# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.255.13/10576/1 Dashboard: http://10.20.255.13:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1049,Kevin,0.9889387168849251,-0.18879907519645345
2000-01-01 00:00:01,987,Zelda,0.18435140106734704,0.22328546191503462
2000-01-01 00:00:02,1032,Oliver,-0.8656486136303023,-0.84347660766973
2000-01-01 00:00:03,985,George,0.1531226882077772,0.8129151716586236
2000-01-01 00:00:04,999,Ingrid,0.6381471974881376,-0.03373012435495837
2000-01-01 00:00:05,1035,Tim,0.7300453791495161,0.6402288571435129
2000-01-01 00:00:06,1027,Kevin,-0.8761155539185899,-0.6234723158090154
2000-01-01 00:00:07,1031,Kevin,0.4603115993580653,-0.34967382626203114
2000-01-01 00:00:08,1031,George,-0.45683736345572634,-0.5874202378045148


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1062,Zelda,-0.0102339469344217,-0.5690278182545085
2000-01-30 00:00:01,935,Victor,0.1151823769590572,0.6024583797219896
2000-01-30 00:00:02,994,Dan,-0.8864102911848004,-0.9999383764723666
2000-01-30 00:00:03,961,Edith,0.5748144180731933,0.6834480724465504
2000-01-30 00:00:04,1029,Ingrid,0.6147553212554586,0.5790933853868752
2000-01-30 00:00:05,1010,Yvonne,0.05119278174257258,-0.8951626316990005
2000-01-30 00:00:06,968,Jerry,-0.9600717751495635,-0.269244543073347
2000-01-30 00:00:07,1040,Quinn,0.3163826788626056,-0.9175325840279289
2000-01-30 00:00:08,943,Zelda,-0.8013483162148571,0.03132276471432305


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1049,Kevin,0.988939,-0.188799
1,2000-01-01 00:00:01,987,Zelda,0.184351,0.223285
2,2000-01-01 00:00:02,1032,Oliver,-0.865649,-0.843477
3,2000-01-01 00:00:03,985,George,0.153123,0.812915
4,2000-01-01 00:00:04,999,Ingrid,0.638147,-0.033730


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1049,Kevin,0.988939,-0.188799
1,2000-01-01 00:00:01,987,Zelda,0.184351,0.223285
2,2000-01-01 00:00:02,1032,Oliver,-0.865649,-0.843477
3,2000-01-01 00:00:03,985,George,0.153123,0.812915
4,2000-01-01 00:00:04,999,Ingrid,0.638147,-0.033730


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.66 s, sys: 715 ms, total: 5.37 s
Wall time: 3.51 s


name
Alice      -0.001492
Bob        -0.000668
Charlie     0.001509
Dan        -0.001079
Edith       0.003000
Frank      -0.000993
George      0.000419
Hannah     -0.000839
Ingrid      0.000143
Jerry      -0.000085
Kevin       0.000847
Laura       0.000973
Michael    -0.000197
Norbert    -0.001075
Oliver      0.001497
Patricia   -0.001590
Quinn      -0.000893
Ray         0.003605
Sarah      -0.001304
Tim        -0.000583
Ursula     -0.000104
Victor      0.000354
Wendy       0.000324
Xavier      0.000493
Yvonne      0.002818
Zelda       0.002040
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.5 s, sys: 156 ms, total: 1.66 s
Wall time: 1.14 s


name
Alice      -0.001492
Bob        -0.000668
Charlie     0.001509
Dan        -0.001079
Edith       0.003000
Frank      -0.000993
George      0.000419
Hannah     -0.000839
Ingrid      0.000143
Jerry      -0.000085
Kevin       0.000847
Laura       0.000973
Michael    -0.000197
Norbert    -0.001075
Oliver      0.001497
Patricia   -0.001590
Quinn      -0.000893
Ray         0.003605
Sarah      -0.001304
Tim        -0.000583
Ursula     -0.000104
Victor      0.000354
Wendy       0.000324
Xavier      0.000493
Yvonne      0.002818
Zelda       0.002040
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.42 s, sys: 58.2 ms, total: 1.48 s
Wall time: 1.08 s


name
Alice      -0.001492
Bob        -0.000668
Charlie     0.001509
Dan        -0.001079
Edith       0.003000
Frank      -0.000993
George      0.000419
Hannah     -0.000839
Ingrid      0.000143
Jerry      -0.000085
Kevin       0.000847
Laura       0.000973
Michael    -0.000197
Norbert    -0.001075
Oliver      0.001497
Patricia   -0.001590
Quinn      -0.000893
Ray         0.003605
Sarah      -0.001304
Tim        -0.000583
Ursula     -0.000104
Victor      0.000354
Wendy       0.000324
Xavier      0.000493
Yvonne      0.002818
Zelda       0.002040
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html